# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 66 new papers today
          33 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/32 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.03872


extracting tarball to tmp_2212.03872...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.03875


extracting tarball to tmp_2212.03875...

...

/tmp/ipykernel_2037/4030337529.py:34: LatexWarning: 2212.04480 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2212.04485


extracting tarball to tmp_2212.04485...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2212.04485/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'myDefs.tex' from 'tmp_2212.04485/myDefs.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04026-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04026) | **The JWST UNCOVER Treasury survey: Ultradeep NIRSpec and NIRCam  ObserVations before the Epoch of Reionization**  |
|| Rachel Bezanson, et al. -- incl., <mark>Natascha Forster Schreiber</mark>, <mark>Anna de Graaff</mark>, <mark>Jenny E. Greene</mark> |
|*Appeared on*| *2022-12-09*|
|*Comments*| *17 pages, 8 figures, 4 tables, submitted to ApJ, comments welcome*|
|**Abstract**| In this paper we describe the survey design for the Ultradeep NIRSpec and NIRCam ObserVations before the Epoch of Reionization (UNCOVER) Cycle 1 JWST Treasury program, which executed its early imaging component in November 2022. The UNCOVER survey includes ultradeep ($\sim29-30\mathrm{AB}$) imaging of $\sim$45 arcmin$^2$ on and around the well-studied Abell 2744 galaxy cluster at $z=0.308$ and will follow-up ${\sim}500$ galaxies with extremely deep low-resolution spectroscopy with the NIRSpec/PRISM during the summer of 2023. We describe the science goals, survey design, target selection, and planned data releases. We also present and characterize the depths of the first NIRCam imaging mosaic, highlighting previously unparalleled resolved and ultradeep 2-4 micron imaging of known objects in the field. The UNCOVER primary NIRCam mosaic spans 28.8 arcmin$^2$ in seven filters (F115W, F150W, F200W, F277W, F356W, F410M, F444W) and 16.8 arcmin$^2$ in our NIRISS parallel (F115W, F150W, F200W, F356W, and F444W). To maximize early community use of the Treasury data set, we publicly release full reduced mosaics of public JWST imaging including 45 arcmin$^2$ NIRCam and 17 arcmin$^2$ NIRISS mosaics on and around the Abell 2744 cluster, including the Hubble Frontier Field primary and parallel footprints. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03981-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03981) | **The wide-field, multiplexed, spectroscopic facility WEAVE: Survey  design, overview, and simulated implementation**  |
|| Shoko Jin, et al. -- incl., <mark>D. J. B. Smith</mark>, <mark>Lilian Domínguez-Palmero</mark>, <mark>Kelley M. Hess</mark>, <mark>Jim Lewis</mark> |
|*Appeared on*| *2022-12-09*|
|*Comments*| *41 pages, 27 figures, accepted for publication by MNRAS*|
|**Abstract**| WEAVE, the new wide-field, massively multiplexed spectroscopic survey facility for the William Herschel Telescope, will see first light in late 2022. WEAVE comprises a new 2-degree field-of-view prime-focus corrector system, a nearly 1000-multiplex fibre positioner, 20 individually deployable 'mini' integral field units (IFUs), and a single large IFU. These fibre systems feed a dual-beam spectrograph covering the wavelength range 366$-$959\,nm at $R\sim5000$, or two shorter ranges at $R\sim20\,000$. After summarising the design and implementation of WEAVE and its data systems, we present the organisation, science drivers and design of a five- to seven-year programme of eight individual surveys to: (i) study our Galaxy's origins by completing Gaia's phase-space information, providing metallicities to its limiting magnitude for $\sim$3 million stars and detailed abundances for $\sim1.5$ million brighter field and open-cluster stars; (ii) survey $\sim0.4$ million Galactic-plane OBA stars, young stellar objects and nearby gas to understand the evolution of young stars and their environments; (iii) perform an extensive spectral survey of white dwarfs; (iv) survey $\sim400$ neutral-hydrogen-selected galaxies with the IFUs; (v) study properties and kinematics of stellar populations and ionised gas in $z<0.5$ cluster galaxies; (vi) survey stellar populations and kinematics in $\sim25\,000$ field galaxies at $0.3\lesssim z \lesssim 0.7$; (vii) study the cosmic evolution of accretion and star formation using $>1$ million spectra of LOFAR-selected radio sources; (viii) trace structures using intergalactic/circumgalactic gas at $z>2$. Finally, we describe the WEAVE Operational Rehearsals using the WEAVE Simulator. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.03872-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.03872) | **On the Application of Bayesian Leave-One-Out Cross-Validation to  Exoplanet Atmospheric Analysis**  |
|| Luis Welbanks, Peter McGill, <mark>Michael Line</mark>, Nikku Madhusudhan |
|*Appeared on*| *2022-12-09*|
|*Comments*| *Accepted for publication in The Astronomical Journal*|
|**Abstract**| Over the last decade, exoplanetary transmission spectra have yielded an unprecedented understanding about the physical and chemical nature of planets outside our solar system. Physical and chemical knowledge is mainly extracted via fitting competing models to spectroscopic data, based on some goodness-of-fit metric. However, current employed metrics shed little light on how exactly a given model is failing at the individual data point level and where it could be improved. As the quality of our data and complexity of our models increases, there is an urgent need to better understand which observations are driving our model interpretations. Here we present the application of Bayesian leave-one-out cross-validation to assess the performance of exoplanet atmospheric models and compute the expected log pointwise predictive density (elpd$_\text{LOO}$). elpd$_\text{LOO}$ estimates the out-of-sample predictive accuracy of an atmospheric model at data point resolution providing interpretable model criticism. We introduce and demonstrate this method on synthetic HST transmission spectra of a hot Jupiter. We apply elpd$_\text{LOO}$ to interpret current observations of HAT-P-41b and assess the reliability of recent inferences of H$^-$ in its atmosphere. We find that previous detections of H$^{-}$ are dependent solely on a single data point. This new metric for exoplanetary retrievals complements and expands our repertoire of tools to better understand the limits of our models and data. elpd$_\text{LOO}$ provides the means to interrogate models at the single data point level, a prerequisite for robustly interpreting the imminent wealth of spectroscopic information coming from JWST. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04455-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04455) | **XMM-Newton and Chandra observations of the candidate Fermi-LAT pulsar  4FGL J1015.5-6030**  |
|| Jeremy Hare, et al. -- incl., <mark>George G. Pavlov</mark> |
|*Appeared on*| *2022-12-09*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**| 4FGL J1015.5-6030 is an unidentified Fermi-LAT source hosting a bright, extended X-ray source whose X-ray spectrum is consistent with that of a young pulsar, yet no pulsations have been found. Here we report on XMM-Newton timing and Chandra imaging observations of the X-ray counterpart of 4FGL J1015.5-6030. We find no significant periodicity from the source and place a 3$\sigma$ upper-limit on its pulsed fraction of 34$\%$. The Chandra observations resolve the point source from the extended emission. We find that the point source's spectrum is well fit by a blackbody model, with temperature $kT=0.205\pm0.009$ keV, plus a weak power-law component, which is consistent with a thermally emitting neutron star with a magnetospheric component. The extended emission spans angular scales of a few arcseconds up to about 30$''$ from the point source and its spectrum is well fit by a power-law model with a photon index $\Gamma=1.70\pm0.05$. The extended emission's spectrum and 0.5-10 keV luminosity of 4$\times10^{32}$ erg s$^{-1}$ (at a plausible distance of 2 kpc) are consistent with that of a pulsar wind nebula. Based on a comparison to other GeV and X-ray pulsars, we find that this putative pulsar is likely a middle-aged (i.e., $\tau\sim 0.1$--1 Myr) radio-quiet pulsar with $\dot{E}\sim10^{34}-10^{35}$ erg s$^{-1}$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04485-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04485) | **A roadmap to cosmological parameter analysis with third-order shear  statistics II: Analytic covariance estimate**  |
|| <mark>Laila Linke</mark>, <mark>Sven Heydenreich</mark>, Pierre A. Burger, Peter Schneider |
|*Appeared on*| *2022-12-09*|
|*Comments*| *21 pages + appendix, 15 figures, 1 table; submitted to Astronomy & Astrophysics*|
|**Abstract**| Third-order weak lensing statistics are a promising tool for cosmological analyses since they extract cosmological information in the non-Gaussianity of the cosmic large-scale structure. However, such analyses require precise and accurate models for the covariance. In this second paper of a series on third-order weak lensing statistics, we derive and validate an analytic model for the covariance of the third-order aperture statistics $\langle M_\mathrm{ap}^3\rangle$. We derive the covariance model from a real-space estimator for $\langle M_\mathrm{ap}^3\rangle$. We validate the model by comparing it to estimates from simulated Gaussian random fields (GRF) and two sets of N-body simulations. Finally, we perform mock cosmological analyses with the model covariance and the simulation estimate to compare the resulting parameter constraints. We find good agreement between the model and the simulations, both for the GRF and the $N$-body simulations. The figure-of-merit in the $S_8$-$\Omega_\mathrm{m}$ plane from our covariance model is within 3\% of the one obtained from the simulated covariances. We also show that our model, which is based on an estimator using convergence maps, can be used to obtain upper and lower bounds for the covariance of an estimator based on three-point shear correlation functions. This second estimator is required for realistic survey data. In our derivation, we find that the $\langle M_\mathrm{ap}^3\rangle$ covariance cannot be obtained from the bispectrum covariance and that it includes several `finite-field terms' that do not scale with the inverse survey area. Our covariance model is sufficiently accurate for analysing stage III surveys. Covariances for statistics in Fourier space cannot always be straightforwardly converted into covariance for real-space statistics. The modelling code is available at https://github.com/sheydenreich/threepoint/releases/ . |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04151-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04151) | **Photometric and Structural Parameters of Newly Discovered Nuclear Star  Clusters in Local Volume Galaxies**  |
|| Nils Hoyer, et al. -- incl., <mark>Nadine Neumayer</mark>, <mark>Jenny E. Greene</mark> |
|*Appeared on*| *2022-12-09*|
|*Comments*| *19 pages, 12 figures, 5 tables. Resubmitted to MNRAS*|
|**Abstract**| We use high-resolution Hubble Space Telescope imaging data of dwarf galaxies in the Local Volume ($\lesssim 11$ Mpc) to parameterise 19 newly discovered nuclear star clusters (NSCs). Most of the clusters have stellar masses of $M_{\star}^{nsc} \lesssim 10^6$ M$_{\odot}$ and compare to Galactic globular clusters in terms of ellipticity, effective radius, stellar mass, and surface density. The clusters are modelled with a S\'ersic profile and their surface brightness evaluated at the effective radius reveals a tight positive correlation to the host galaxy stellar mass. Our data also indicate an increase in slope of the density profiles with increasing mass, perhaps indicating an increasing role for in-situ star formation in more massive hosts. We evaluate the scaling relation between the clusters and their host galaxy stellar mass to find an environmental dependence: for NSCs in field galaxies, the slope of the relation is $\alpha = 0.82^{+0.08}_{-0.08}$ whereas $\alpha = 0.55^{+0.06}_{-0.05}$ for dwarfs in the core of the Virgo cluster. Restricting the fit for the cluster to $M_{\star}^{gal} \geq 10^{6.5}$ M$_{\odot}$ yields $\alpha = 0.70^{+0.08}_{-0.07}$, in agreement with the field environment within the $1\sigma$ interval. The environmental dependence is due to the lowest-mass nucleated galaxies and we speculate that this is either due to an increased number of progenitor globular clusters merging to become an NSC, or due to the formation of more massive globular clusters in dense environments, depending on the initial globular cluster mass function. Our results clearly corroborate recent results in that there exists a tight connection between NSCs and globular clusters in dwarf galaxies. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure density_mass.pdf</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04480-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04480) | **Discovery and properties of the earliest galaxies with confirmed  distances**  |
|| B. E. Robertson, et al. -- incl., <mark>H.-W. Rix</mark>, <mark>A. de Graaff</mark> |
|*Appeared on*| *2022-12-09*|
|*Comments*| *Submitted November 17, 2022*|
|**Abstract**| Surveys with James Webb Space Telescope (JWST) have discovered candidate galaxies in the first 400 Myr of cosmic time. The properties of these distant galaxies provide initial conditions for understanding early galaxy formation and cosmic reionisation. Preliminary indications have suggested these candidate galaxies may be more massive and abundant than previously thought. However, without spectroscopic confirmation of their distances to constrain their intrinsic brightnesses, their inferred properties remain uncertain. Here we report on four galaxies located in the JWST Advanced Deep Extragalactic Survey (JADES) Near-Infrared Camera (NIRCam) imaging with photometric redshifts $z\sim10-13$ subsequently confirmed by JADES JWST Near- Infrared Spectrograph (NIRSpec) observations. These galaxies include the first redshift $z>12$ systems both discovered and spectroscopically confirmed by JWST. Using stellar population modelling, we find the galaxies typically contain a hundred million solar masses in stars, in stellar populations that are less than one hundred million years old. The moderate star formation rates and compact sizes suggest elevated star formation rate surface densities, a key indicator of their formation pathways. Taken together, these measurements show that the first galaxies contributing to cosmic reionisation formed rapidly and with intense internal radiation fields. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.04452-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.04452) | **The Pan-STARRS1 z>5.6 quasar survey II: Discovery of 55 Quasars at  5.6<z<6.5**  |
|| Eduardo Banados, et al. -- incl., <mark>Romain A. Meyer</mark>, <mark>Fabian Walter</mark>, <mark>Joseph F. Hennawi</mark>, <mark>Yana Khusanova</mark>, <mark>Hans-Walter Rix</mark>, <mark>Joseph Simon</mark>, <mark>Zhang-Liang Xie</mark> |
|*Appeared on*| *2022-12-09*|
|*Comments*| *Version after addressing referee report. See companion paper by Schindler et al*|
|**Abstract**| The identification of bright quasars at z>6 enables detailed studies of supermassive black holes, massive galaxies, structure formation, and the state of the intergalactic medium within the first billion years after the Big Bang. We present the spectroscopic confirmation of 55 quasars at redshifts 5.6<z<6.5 and UV magnitudes -24.5<M1450<-28.5 identified in the optical Pan-STARRS1 and near-IR VIKING surveys (48 and 7, respectively). Five of these quasars have been independently discovered in other studies. The quasar sample shows an extensive range of physical properties, including 17 objects with weak emission lines, ten broad absorption line quasars, and five with strong radio emission (radio-loud quasars). There are also a few notable sources in the sample, including a blazar candidate at z=6.23, a likely gravitationally lensed quasar at z=6.41, and a z=5.84 quasar in the outskirts of the nearby (D~3 Mpc) spiral galaxy M81. The blazar candidate remains undetected in NOEMA observations of the [CII] and underlying emission, implying a star-formation rate <30-70 Msun/yr. A significant fraction of the quasars presented here lies at the foundation of the first measurement of the z~6 quasar luminosity function from Pan-STARRS1 (introduced in a companion paper). The quasars presented here will enable further studies of the high-redshift quasar population with current and future facilities. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error sequence item 0: expected str instance, TexNode found</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.04026.md
    + _build/html/tmp_2212.04026/./figures/magnification_HFF.png
    + _build/html/tmp_2212.04026/./figures/depth_area.png
    + _build/html/tmp_2212.04026/./figures/Footprints.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\minus}{\scalebox{0.5}[1.0]{-}}$
$\newcommand{\}{natexlab}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\url$}[1]{\href{#1}{#1}}$
$\newcommand{$\dodoi$}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{$\doeprint$}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{$\doarXiv$}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\minus$}{\scalebox{0.5}[1.0]{-}}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

#  NIRSpec and NIRCam ObserVations before the Epoch of Reionization

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.04026-b31b1b.svg)](https://arxiv.org/abs/2212.04026)<mark>Appeared on: 2022-12-09</mark> - _17 pages, 8 figures, 4 tables, submitted to ApJ, comments welcome_

</div>
<div id="authors">

Rachel Bezanson, et al. -- incl., <mark><mark>Natascha Forster Schreiber</mark></mark>, <mark><mark>Anna de Graaff</mark></mark>, <mark><mark>Jenny E. Greene</mark></mark>

</div>
<div id="abstract">

**Abstract:** In this paper we describe the survey design for the Ultradeep NIRSpec and NIRCam ObserVations before the Epoch of Reionization (UNCOVER) Cycle 1\JWSTTreasury program, which executed its early imaging component in November 2022. The UNCOVER survey includes ultradeep ($\sim29-30\mathrm{AB}$) imaging of$\sim$45 arcmin$^2$on and around the well-studied Abell 2744 galaxy cluster at$z=0.308$and will follow-up${\sim}500$galaxies with extremely deep low-resolution spectroscopy with the NIRSpec/PRISM during the summer of 2023. We describe the science goals, survey design, target selection, and planned data releases. We also present and characterize the depths of the first NIRCam imaging mosaic, highlighting previously unparalleled resolved and ultradeep 2-4 micron imaging of known objects in the field. The UNCOVER primary NIRCam mosaic spans 28.8 arcmin$^2$in seven filters (F115W, F150W, F200W, F277W, F356W, F410M, F444W) and 16.8 arcmin$^2$in our NIRISS parallel (F115W, F150W, F200W, F356W, and F444W).  To maximize early community use of the Treasury data set, we publicly release full reduced mosaics of public\JWSTimaging including 45 arcmin$^2$NIRCam and 17 arcmin$^2$NIRISS mosaics on and around the Abell 2744 cluster, including the Hubble Frontier Field primary and parallel footprints.

</div>

<div id="div_fig1">

<img src="tmp_2212.04026/./figures/magnification_HFF.png" alt="Fig2" width="100%"/>

**Figure 2. -** Gravitational lensing magnification contours in Abell 2744 are extremely extended, due to the complex structure of the multiple cluster cores. The above curves are taken from the CATS v4.1 \citet{jauzac:15} model, originally used for our proposal. The UNCOVER NIRCam mosaic (dark purple) spans the $\mu=2$ curve, which is significantly larger than the Hubble Frontier Field (black dashed outline). By extending to the northern subclumps and the UNCOVER mosaic will enable a more detailed mapping of that region. The UNCOVER-based lens model will be published soon (L. Furtak et al. in preparation).  (*fig:magnification*)

</div>
<div id="div_fig2">

<img src="tmp_2212.04026/./figures/depth_area.png" alt="Fig1" width="100%"/>

**Figure 1. -** The UNCOVER combined imaging (purple star) probes a unique regime in the context of \HST extragalactic ultradeep fields (blue circles) and \JWST Cycle 1 imaging surveys (red squares); without lensing it probes deeper than previous \HST surveys and wide field programs. Gravitational lensing (approximate lensing vectors indicated by dashed arrows) from the Abell 2744 allows UNCOVER imaging to probe the intrinsically faintest objects of any \JWST project in Cycle 1. We note that lensing vectors are approximate, as the UNCOVER survey includes lensed and unlensed areas of Abell 2744. (*fig:deptharea*)

</div>
<div id="div_fig3">

<img src="tmp_2212.04026/./figures/Footprints.png" alt="Fig3" width="100%"/>

**Figure 3. -** The Abell 2744 cluster has extensive deep optical/NIR (\HST and \emph{VLT}/MUSE) and \JWST coverage. The \JWST/UNCOVER footprints (purple) along with \HST imaging (blue) and \emph{VLT}/MUSE deep datacube (green, center) and \JWST Cycle 1 imaging and spectroscopy (red, right) against imaging from the Legacy Imaging survey \citep{dey:19}. In the left panel, we highlight the detailed layout of the UNCOVER dataset, differentiating between the first epoch (dark purple, NIRCam primary imaging and NIRISS parallel imaging) and second epoch (planned NIRSpec spectroscopy and NIRCam parallel imaging, light purple). The NIRCam mosaic is designed to match the $\mu=2$ magnification curve (see Fig.\ref{fig:magnification}). The footprints of the NIRSpec spectroscopy are provisional and subject to target selection. (*fig:footprints*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

130  publications in the last 7 days.
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/html/2211.16996.md
	 _build/html/2211.16510.md
	 _build/html/2211.16123.md
	 _build/html/2211.15816.md
	 _build/html/2211.15698.md
	 _build/html/2211.15576.md
	 _build/html/2211.15434.md
	 _build/html/2211.15288.md
	 _build/html/2211.15224.md
	 _build/html/2211.14543.md
	 _build/html/2211.14330.md
	 _build/html/2211.14322.md
	 _build/html/2211.14132.md
	 _build/html/2211.13811.md
	 _build/html/2211.13426.md
	 _build/html/2211.13318.md
	 _build/html/2211.13146.md
	 _build/html/2211.12613.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers